In [ ]:
#This is for Know the satellites position with a specific time and date 

In [7]:
#Now I need to know the near 
import datetime
import time
import os, sys
from ftplib import FTP
from pykml import parser
import keytree
import re
import pandas as pd
import numpy as np
from pandas import ExcelWriter

In [8]:
def to_radians(array):
    rads = array*(np.pi /180.)
    return rads

def compute_distance(df,longitude, latitude):
    R = 6371e3 # earth radius metres, mean radius = 6371km
    n = len(df.Latitude)
    ArrayLat = pd.Series([latitude] * n)
    ArrayLon = pd.Series([longitude] * n)
    #print len(ArrayLat), type(ArrayLat), type(df.Latitude)
    phi1 = to_radians(df.Latitude) #degrees to radians
    phi2 = to_radians(ArrayLat)
    lambda1 = to_radians(df.Longitude)
    lambda2 = to_radians(ArrayLon)
    dphi = phi2 - phi1
    dlambda = lambda2 - lambda1
    a = np.sin(dphi/2.)**2 + (np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2.)**2 )
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1.-a))
    d = R * c
    return d

In [9]:
Sat_Time = "01:20:23"
Sat_Date = "10/01/2017"
Longitude_inGround = -73.1224429 
Latitude_inGround = 7.1388027


In [10]:
ArrayLat = [Longitude_inGround] * 4
test = pd.Series(ArrayLat)
print len(ArrayLat),type(ArrayLat),type(test)

4 <type 'list'> <class 'pandas.core.series.Series'>


In [11]:
#Parse String to Date
date_sat = datetime.datetime.strptime(Sat_Date,'%d/%m/%Y')
date_sat.day,date_sat.month,date_sat.year

(10, 1, 2017)

In [12]:
day_of_year = date_sat.timetuple().tm_yday
day_of_year

10

In [13]:
#Parse String to Time
time_sat = datetime.datetime.strptime(Sat_Time,'%H:%M:%S')
time_sat.hour,time_sat.minute,time_sat.second

(1, 20, 23)

In [14]:
UNAVCO_list = pd.read_csv("UNAVCOlist.csv")
UNAVCO_list.columns

Index([u'Unnamed: 0', u'CODE', u'Longitude', u'Latitude'], dtype='object')

search the nearest station

In [15]:
#Universidad Industrial de Santander
Latitude_inGround,Longitude_inGround

(7.1388027, -73.1224429)

In [16]:
d = compute_distance(UNAVCO_list,Longitude_inGround,Latitude_inGround)
UNAVCO_list['Distance'] = d

In [17]:
UNAVCO_list.sort_values(['Distance'], ascending=[True], inplace=True)
UNAVCO_list

,Unnamed: 0,CODE,Longitude,Latitude,Distance
2632,2632,BUCM,-73.181700,7.116900,6977.043272
2673,2673,BUCM,-73.181700,7.116900,6977.043272
2901,2901,BUCM,-73.181700,7.116900,6977.043272
2690,2690,FLOR,-73.088600,7.041600,11435.389590
2668,2668,ARAT,-73.052200,6.703200,49053.488836
3048,3048,PLON,-72.649000,7.384000,58911.122695
25086,25086,MECE,-73.712000,7.107200,65144.656963
2728,2728,SOCO,-73.241700,6.462500,76345.500651
2871,2871,ABRE,-73.225900,8.086800,106027.388816
24149,24149,CUC1,-72.512800,7.932300,110911.258013


In [18]:
#search the nearest land stage
IDmin = UNAVCO_list.Distance.argmin()
S_Code = UNAVCO_list.CODE[IDmin]
S_Dist = UNAVCO_list.Distance[IDmin]
print S_Code,UNAVCO_list.Longitude[IDmin],UNAVCO_list.Latitude[IDmin],S_Dist

BUCM -73.1817 7.1169 6977.04327191


Now I have the Name of the Lan Station that I need.


In [19]:
UNAVCO_FTP = 'data-out.unavco.org'
NASA_FTP = 'cddis.gsfc.nasa.gov'

f = FTP(UNAVCO_FTP)
#print "Welcome:", f.getwelcome()
f.login()

#Now I wnat to look the folder that I need.
FTP_path = '/pub/rinex/nav/' + str(date_sat.year) + '/' + str(day_of_year).zfill(3)

#[data-out.unavco.org] --> /pub/rinex/nav/2017/002
#[cddis.gsfc.nasa.gov] --> /pub/gps/data/daily/2017/001/17n --> 
#    FTP_path ='/pub/gps/data/daily/'+ str_year + '/' + str(day_of_year).zfill(3) + '/' + str_year[2:] + 'n'

print "Folder Needed:", f.cwd(FTP_path)
print "Current working directory:", f.pwd()

Folder Needed: 250 OK. Current directory is /pub/rinex/nav/2017/010
Current working directory: /pub/rinex/nav/2017/010


In [ ]:
LineData = 0
while (1):
    try:
        IDmin = UNAVCO_list.Distance.argmin()
        NameStation = UNAVCO_list.CODE[IDmin]
        
        NameFile = NameStation.lower() + str(day_of_year).zfill(3) + '0.' + str(date_sat.year)[2:] + 'n.Z'     #bcwr 0020 .17n.Z
        print("Station: %s" % (NameFile))
        f.voidcmd("TYPE I")
        datasock, size = f.ntransfercmd("RETR " + NameFile)
        bytes_so_far = 0
        fd = open(NameFile, 'wb')
        while 1:
            buf = datasock.recv(2048)
            if not buf:
                break
            fd.write(buf)
            bytes_so_far += len(buf)
            print "\rReceived", bytes_so_far,
            if size:
                print "of %d total bytes (%.1f%%)" % (
                    size, 100 * bytes_so_far / float(size)),
            else:
                print "bytes",
            sys.stdout.flush()
            if bytes_so_far == len(buf):
                print ("%.2f" % str(UNAVCO_list.Distance[IDmin]/1000)) + "[Km]"
                break
        print
        fd.close()
        datasock.close()
        f.voidresp
        break
    except:
        UNAVCO_list = UNAVCO_list[UNAVCO_list.CODE != NameStation]

Station: bucm0100.17n.Z
Station: flor0100.17n.Z
Station: arat0100.17n.Z
Station: plon0100.17n.Z
Station: mece0100.17n.Z
Station: soco0100.17n.Z
Station: abre0100.17n.Z
Station: cuc10100.17n.Z
Station: sard0100.17n.Z
Station: oliv0100.17n.Z
Station: achi0100.17n.Z
Station: moni0100.17n.Z
Station: paip0100.17n.Z
Station: pber0100.17n.Z
Station: tame0100.17n.Z
Station: tibu0100.17n.Z
Station: tunj0100.17n.Z
Station: aqui0100.17n.Z
Station: hato0100.17n.Z
Station: paza0100.17n.Z
Station: paut0100.17n.Z
Station: burr0100.17n.Z
Station: vequ0100.17n.Z
Station: capi0100.17n.Z
Station: yopa0100.17n.Z
Station: mntr0100.17n.Z
Station: azul0100.17n.Z
Station: taur0100.17n.Z
Station: roqe0100.17n.Z
Station: sesq0100.17n.Z
Station: slpa0100.17n.Z
Station: cauc0100.17n.Z
Station: yaru0100.17n.Z
Station: tara0100.17n.Z
Station: barb0100.17n.Z
Station: arau0100.17n.Z
Station: osos0100.17n.Z
Station: cl-10100.17n.Z
Station: hond0100.17n.Z
Station: cart0100.17n.Z
Station: chia0100.17n.Z
Station: bogo010